# Import

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
import math
import numpy as np

In [ ]:
def detect_botnet(label):
    if 'Botnet' in label:
        return 0
    else:
        return 1

#create a column with 0 if botnet and 1 else
    
def transform_date(strdate):
    first_split = strdate.split('/')
    second_split = first_split[-1].split(' ')
    third_split = second_split[-1].split(':')
    four_split = third_split[-1].split('.')
    year = int(first_split[0])
    month = int(first_split[1])
    day = int(second_split[0])
    hour = int(third_split[0])
    minute = int(third_split[1])
    second = int(four_split[0])
    date = datetime(year = year, month = month, day = day, hour = hour, minute = minute, second = second)
    return date.timestamp()

#transform the date in a numerical value

def detect_anormal(vec):
    ect  = np.std(u1)
    mean = np.mean(u1)
    return max(abs(u1)) > mean + 3 * ect

#detect anormal traffic according to the mean and standard deviation

# Download of data

If use on colab, you can download data with this command (data server is down for the moment)

In [ ]:
!wget https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-42/detailed-bidirectional-flow-labels/capture20110810.binetflow
df = pd.read_csv('/content/capture20110810.binetflow')

--2022-03-18 15:30:34--  https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-42/detailed-bidirectional-flow-labels/capture20110810.binetflow
Resolving mcfp.felk.cvut.cz (mcfp.felk.cvut.cz)... 147.32.82.194
Connecting to mcfp.felk.cvut.cz (mcfp.felk.cvut.cz)|147.32.82.194|:443... failed: No route to host.


If working on local, put Binetflow file on the same directory than the notebook. I recommend doing it and working on local since it is quite slow to import one dataset on Colab.

In [ ]:
df = pd.read_csv('capture20110817.binetflow')
#df = pd.read_csv('capture20110810.binetflow')
#df = pd.read_csv('capture20110811.binetflow')

# Analyze of the data

small extract of the dataset

In [ ]:
df.head()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/10 09:46:59.607825,1.026539,tcp,94.44.127.113,1577,->,147.32.84.59,6881,S_RA,0.0,0.0,4,276,156,flow=Background-Established-cmpgw-CVUT
1,2011/08/10 09:47:00.634364,1.009595,tcp,94.44.127.113,1577,->,147.32.84.59,6881,S_RA,0.0,0.0,4,276,156,flow=Background-Established-cmpgw-CVUT
2,2011/08/10 09:47:48.185538,3.056586,tcp,147.32.86.89,4768,->,77.75.73.33,80,SR_A,0.0,0.0,3,182,122,flow=Background-TCP-Attempt
3,2011/08/10 09:47:48.230897,3.111769,tcp,147.32.86.89,4788,->,77.75.73.33,80,SR_A,0.0,0.0,3,182,122,flow=Background-TCP-Attempt
4,2011/08/10 09:47:48.963351,3.083411,tcp,147.32.86.89,4850,->,77.75.73.33,80,SR_A,0.0,0.0,3,182,122,flow=Background-TCP-Attempt


interesting info of some numeric value

In [ ]:
df.describe()

,Dur,sTos,dTos,TotPkts,TotBytes,SrcBytes
count,2.824636e+06,2.814046e+06,2.629446e+06,2.824636e+06,2.824636e+06,2.824636e+06
mean,4.322060e+02,2.786664e-02,4.430591e-04,2.547990e+01,1.929134e+04,4.155290e+03
std,9.958423e+02,2.220372e+00,3.332693e-02,3.124788e+03,3.212186e+06,1.858530e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00
25%,3.130000e-04,0.000000e+00,0.000000e+00,2.000000e+00,2.140000e+02,7.800000e+01
50%,1.194000e-03,0.000000e+00,0.000000e+00,2.000000e+00,2.610000e+02,8.300000e+01
75%,9.012607e+00,0.000000e+00,0.000000e+00,4.000000e+00,5.880000e+02,2.610000e+02
max,3.600031e+03,1.920000e+02,3.000000e+00,2.686731e+06,2.689640e+09,2.635366e+09


We see that the label are separated in 3 class (Background, Normal and Botnet)

In [ ]:
df['Label'].unique()

array(['flow=Background-Established-cmpgw-CVUT',
       'flow=Background-TCP-Attempt', 'flow=Background-TCP-Established',
       'flow=Background-Attempt-cmpgw-CVUT',
       'flow=To-Background-CVUT-Proxy', 'flow=Background-UDP-Established',
       'flow=Background', 'flow=From-Background-CVUT-Proxy',
       'flow=To-Background-UDP-CVUT-DNS-Server',
       'flow=Background-UDP-Attempt',
       'flow=From-Normal-V42-UDP-CVUT-DNS-Server',
       'flow=Background-UDP-NTP-Established-1',
       'flow=To-Background-CVUT-WebServer',
       'flow=Background-google-analytics3',
       'flow=Background-google-analytics2',
       'flow=Background-google-analytics6', 'flow=From-Normal-V42-Jist',
       'flow=Background-google-webmail',
       'flow=Background-google-analytics12',
       'flow=Background-google-analytics1',
       'flow=Background-google-analytics13',
       'flow=Background-google-analytics14', 'flow=To-Background-Jist',
       'flow=Background-ajax.google', 'flow=From-Normal-V42

We create two column (Botnet and Date), for more details go on first part to see the details of the function detect_botnet and transform_date

In [ ]:
df["Botnet"]= df["Label"].apply(detect_botnet)
df["Date"] = df["StartTime"].apply(transform_date)

We separe the dataframe in two dataframe (one normal and other anormal

In [ ]:
df_botnet = df[df['Botnet'] == 0]
df_normal = df[df['Botnet'] == 1]

By focusing on source ip adress, we could easily see a pattern (147.32.84.0/24)

In [ ]:
df_botnet["SrcAddr"].unique() #le 1

array(['147.32.84.165'], dtype=object)

# Implementation of Lakhina Volume

Our implementation is quite different from the original method because of the difference of the dataset: 
- We use a windows time of 600s (10 min)
- We can't separate the u_i in normal and anormal space because according to our results, either all spaces are normal or all spaces are abnormal.
- For simplicity reason, we use a cross validation method to determine the best threshold
- Some ip are not considered, because they don't have enough traffic and it will lead to some error

In [ ]:
threshold = 2500000 #value of the threshold
y_pred = [] #label of prediction
y_true = [] #label of ground truth
false_ip = 1000 #number of ip which are not botnet (we don't use all ip because it is too much time consuming)
window_time = 600

ips = np.concatenate((df_botnet["SrcAddr"].unique(), df_normal["SrcAddr"].unique()[:false_ip]), axis=0)

for c,ip in enumerate(ips):
    df_extract = df[df["SrcAddr"] == ip] #create a dataframe with only one ip source
    first_timestamp = df_extract["Date"].values[0]
    last_timestamp = df_extract["Date"].values[-1]
    diff = (last_timestamp - first_timestamp) / window_time
    if(math.ceil(diff) == math.floor(diff)):
        size = math.ceil(diff) + 1
    else:
        size = math.ceil(diff)

    y = np.zeros((size,3)) #create the matrix y
    if(size > 3):
        for i in df_extract.index:
            index = math.floor((df_extract['Date'][i] - first_timestamp)/ window_time)
            y[index,0] += 1
            y[index,1] += df_extract['TotPkts'][i]
            y[index,2] += df_extract['SrcBytes'][i]

        #complete the matrix y
        
        #apply the pca
        pca = PCA(n_components=3)
        pca.fit(y)
        y_pca = pca.transform(y)
        
        #we define ui according to our pseudo-code
        u1 = y_pca[:,0] / LA.norm(y_pca[:,0])
        u2 = y_pca[:,1] / LA.norm(y_pca[:,1])
        u3 = y_pca[:,2] / LA.norm(y_pca[:,2])
        lst_vec = [u1,u2,u3]
        
        #we count how many subspaces are considered as anormal
        count_number_anormal = 0
        for vec in lst_vec:
            if detect_anormal(vec):
                count_number_anormal+=1
        
        #a ip is considered as a botnet if it subspace are anormal and his norm is superior that the threshold
        if count_number_anormal > 0 and LA.norm(y_pca) > threshold:
            y_pred.append(0)
        else:
            y_pred.append(1)
            
        if 0 in df_extract['Botnet'].values:
            y_true.append(0)
        else:
            y_true.append(1)

# Analyze of result

We see that our confusion matrix is great (False positive near zero and not much of False negative)

In [ ]:
conf = confusion_matrix(y_true, y_pred)
print(conf)

[[  7   3]
 [ 30 634]]


Accuracy and more important balanced accuracy are great, that shows that our method is efficient

In [ ]:
count = sum(sum(conf))
print(f'True Positive: {conf[0,0] / sum(conf[0,:]):.2f}, True Negative: {conf[1,1] / sum(conf[1,:]):.2f}')
print(f'False Positive: {conf[0,1] / sum(conf[0,:]):.2f}, False Negative: {conf[1,0] / sum(conf[1,:]):.2f}')
print(f'Accuracy: {accuracy_score(y_true, y_pred):.2f}, Balanced accuracy: {balanced_accuracy_score(y_true, y_pred):.2f}')

True Positive: 0.70, True Negative: 0.95
False Positive: 0.30, False Negative: 0.05
Accuracy: 0.95, Balanced accuracy: 0.83
